# Analyse et visualisation de données avec Python
## Créer des graphiques avec Altair
Questions
* Comment faire davantage de visualisation en Python?
* Comment faire des graphiques modernes et interactifs?
* Qu'est-ce que la "grammaire des graphiques"?

Objectifs
* Créer un objet `alt.Chart`.
* Créer différents types de graphiques.
* Modifier l'apparence du graphique:
  * Configurer les couleurs.
  * Éditer le titre et le nom des axes.
* Diviser une figure en facettes.
* Sauvegarder une figure en image et en version interactive.

In [ ]:
import numpy as np
import pandas as pd

# Charger et nettoyer les données
surveys_complet = pd.read_csv('../data/surveys.csv').dropna()
surveys_complet

## Pourquoi `altair`? Pourquoi pas `matplotlib`?

Bien que `matplotlib` soit une bibliothèque de visualisation
largement répandue et relativement flexible, la programmation
des graphiques ne suit pas une *grammaire* spécifique.

Ainsi, pour ce chapitre, nous avons fait le choix
de vous présenter la bibliothèque `altair` qui
[facilite la création de graphiques hautement informatifs](https://altair-viz.github.io/index.html)
tout en s'intégrant bien avec Pandas. Le fonctionnement de
`altair` se base sur la grammaire des graphiques interactifs
de [Vega-Lite](https://vega.github.io/vega-lite/),
ce qui rend la programmation à la fois élégante et puissante.

Nous verrons différents concepts de visualisation qui peuvent être
reproduits plus ou moins facilement avec d'autres bibliothèques
telles que `matplotlib`, `plotnine`, `plotly` et `seaborn`.

In [ ]:
import altair as alt

Étant donné que les graphiques générés par Altair ne sont pas que des
images statiques, l'information générée est parfois très lourde et
elle s'accumule dans le notebook s'il y a plusieurs graphiques.
Néanmoins, pour contourner la limite de 5000 lignes dans le DataFrame
de données, on peut désactiver cette limite, à nos risques.

In [ ]:
alt.data_transformers.disable_max_rows()

## Générer des graphiques avec `altair`
Les graphiques `altair` sont construits étape par étape à partir
d'un objet de type `Chart`:
* **Création du graphique** - La première méthode obligatoire débute
  par `mark_`. Par exemple, `mark_point()`.
  À ce stade, c'est normal que tout soit concentré en un point.

In [ ]:
# Création de l'objet Chart et choix du type de graphique
alt.Chart(surveys_complet).mark_point()

* **Encodage des canaux** - La prochaine étape consiste à
  [encoder](https://altair-viz.github.io/user_guide/encodings/)
  des canaux liant certaines variables du DataFrame à divers éléments
  du graphiques. Les principaux paramètres de `encode()` sont :
  `x`, `y`, `color`, `shape` et `size`.

In [ ]:
# Définition des axes ; les points prennent leur position
alt.Chart(surveys_complet).mark_point().encode(
    x=alt.X('hindfoot_length'),
    y=alt.Y('weight'),
)

* **Navigation interactive** - On peut ensuite rendre le graphique
  interactif, ce qui permet de naviguer dans le graphique à l'aide
  de la souris.

In [ ]:
# Permettre les interactions avec la souris
alt.Chart(surveys_complet).mark_point().encode(
    x=alt.X('hindfoot_length'),
    y=alt.Y('weight'),
).interactive()

In [ ]:
# Ajouter du bruit aux longueurs et aux poids
N = len(surveys_complet)
surveys_complet['longueur_bruitee'] = (
    surveys_complet['hindfoot_length'] + np.random.uniform(-0.5, 0.5, N)
)
surveys_complet['poids_bruite'] = (
    surveys_complet['weight'] + np.random.uniform(-0.5, 0.5, N)
)
surveys_complet.columns

* **Affichage interactif des valeurs** - Encoder le canal `tooltip`
  avec une liste de variables à afficher au passage de la souris.

In [ ]:
# Permettre de voir les valeurs associées aux points
graphique = alt.Chart(surveys_complet).mark_point().encode(
    x=alt.X('longueur_bruitee'),
    y=alt.Y('poids_bruite'),
    tooltip=['plot_id', 'species_id', 'hindfoot_length', 'weight'],
)
graphique

* **Sauvegarde du graphique** - C'est possible de le sauvegarder dans le format de notre choix.

In [ ]:
graphique.save('poids_longueur.html')

In [ ]:
try:
    graphique.save('poids_longueur.png')
except BaseException as err:
    print('Erreur:', err)
    print('-> Il vaut mieux utiliser le bouton (•••)')

### Exercice - Créer un histogramme
À partir du DataFrame `surveys_complet`, faites afficher le
décompte du nombre d'enregistrements pour chaque `plot_id`. Instructions :
* Utilisez
  [`mark_bar()`](https://altair-viz.github.io/gallery/simple_bar_chart.html)
  pour générer
  [l'histogramme](https://altair-viz.github.io/gallery/simple_histogram.html)
* Sur l'axe horizontal, spécifiez la variable `'plot_id'` et le
  [type `'ordinal'`](https://altair-viz.github.io/user_guide/encodings/#encoding-data-types)
* Sur l'axe vertical, spécifiez `'count()'` comme variable pour que
  Altair fasse automatiquement le décompte, ce qui évite de passer
  par la méthode `groupby()` du DataFrame

(4 min.)

In [ ]:
alt.Chart(surveys_complet).mark_bar().encode(
    x=alt.X('plot_id').type('ordinal'),
    y=alt.Y('count()'),
)

## Construire un graphique par étapes
Rappel : les éléments de base de tout graphique Altair sont
le `Chart()` avec le DataFrame, un type de graphique `mark_*()`
et des variables utilisées dans `encode()`.

* Ensuite, on commence à modifier le graphique pour en extraire
  davantage d'information. Par exemple, avec de la transparence :

In [ ]:
alt.Chart(surveys_complet).mark_point().encode(
    x=alt.X('longueur_bruitee'),
    y=alt.Y('poids_bruite'),
).configure_mark(
    opacity=0.05,
)

* Pour avoir une couleur différente pour chaque type d'espèce,
  il faut lier la variable `species_id` au canal `color` :

In [ ]:
alt.Chart(surveys_complet).mark_point().encode(
    x=alt.X('longueur_bruitee'),
    y=alt.Y('poids_bruite'),
    color=alt.Color('species_id'),
).configure_mark(
    opacity=0.05,
)

* Étant donné la similarité de certaines couleurs, on peut ensuite
  activer les `tooltip` avec les identifiants de `species_id` :

In [ ]:
alt.Chart(surveys_complet).mark_point().encode(
    x=alt.X('longueur_bruitee'),
    y=alt.Y('poids_bruite'),
    color=alt.Color('species_id'),
    tooltip=['species_id'],
).configure_mark(
    opacity=0.05,
)

* Utiliser une échelle verticale semi-log :

In [ ]:
alt.Chart(surveys_complet).mark_point().encode(
    x=alt.X('longueur_bruitee'),
    y=alt.Y('poids_bruite').scale(type='log', base=2),
    color=alt.Color('species_id'),
    tooltip=['species_id'],
).configure_mark(
    opacity=0.05,
).properties(
    height=384,
)

* Modifier le nom des axes :

In [ ]:
alt.Chart(surveys_complet).mark_point().encode(
    x=alt.X('longueur_bruitee').title('Longueur arrière-pied (mm)'),
    y=alt.Y('poids_bruite').scale(type='log', base=2).title('Poids (g)'),
    color=alt.Color('species_id'),
    tooltip=['species_id'],
).configure_mark(
    opacity=0.05,
).properties(
    height=384,
    title="Poids selon la longueur de l'arrière-pied",
)

### Exercice - Enrichir l'histogramme
Adaptez l'histogramme de l'exercice précédent en associant la variable
`sex` à une échelle de couleurs spécifique :
* L'encodage du canal `color` doit donc se faire avec la variable
  `'sex'`. La méthode `.scale()` permet ensuite d'associer les valeurs
  de domaine `'F'` et `'M'` aux couleurs `'orange'` et `'green'`.
  Voir [un exemple ici](https://altair-viz.github.io/user_guide/customization.html#color-domain-and-range)
* Activez le canal `tooltip` avec `'count()'` pour avoir le décompte
  par sexe

(4 min.)

In [ ]:
alt.Chart(surveys_complet).mark_bar().encode(
    x=alt.X('plot_id').type('ordinal'),
    y=alt.Y('count()'),
    color=alt.Color('sex').scale(
        domain=['F', 'M'],
        range=['orange', 'green'],
    ),
    tooltip=['count()'],
)

## Visualiser des données selon le temps
* Nombre d'enregistrements par type d'espèce pour chaque année :

In [ ]:
alt.Chart(surveys_complet).mark_line().encode(
    x=alt.X('year').type('ordinal'),
    y=alt.Y('count()').scale(type='log', base=2),
    color=alt.Color('species_id'),
)

* Poids médian par type d'espèce pour chaque mois :

In [ ]:
alt.Chart(surveys_complet).mark_line().encode(
    x=alt.X('month').type('ordinal'),
    y=alt.Y('weight').aggregate('median'),
    color=alt.Color('species_id'),
    tooltip=['species_id'],
)

### Exercice - Visualisation selon le temps
`1`. Utilisez la fonction `pd.to_datetime()` pour générer une colonne
     de dates à partir des colonnes `year`, `month` et `day`. (3 min.)

In [ ]:
# Décennie 1990 - pour éviter avril et septembre 2000
dec_1990 = surveys_complet[
    surveys_complet['year'] // 10 == 199].copy()

dec_1990['date'] = pd.to_datetime(dec_1990[['year', 'month', 'day']])
dec_1990['date']

`2`. Affichez le poids médian de chaque espèce selon la `date`.
(3 min.)

In [ ]:
alt.Chart(dec_1990).mark_line().encode(
    x=alt.X('date'),
    y=alt.Y('weight').aggregate('median'),
    color=alt.Color('species_id'),
    tooltip=['species_id', 'date'],
)

## Création de facettes
`altair` a une technique spéciale appelée *création de facettes*
permettant de diviser un graphique en plusieurs sous-graphiques
selon les valeurs d'une variable.

* Avec les différentes valeurs de `sex` :

In [ ]:
alt.Chart(surveys_complet).mark_point().encode(
    x=alt.X('longueur_bruitee'),
    y=alt.Y('poids_bruite').scale(type='log', base=2),
    color=alt.Color('species_id'),
    facet=alt.Facet('sex'),
    tooltip=['species_id'],
).configure_mark(
    opacity=0.05,
).properties(
    width=240,
    height=384,
)

* Avec les nombreuses valeurs de `plot_id` :

In [ ]:
alt.Chart(surveys_complet).mark_point().encode(
    x=alt.X('longueur_bruitee'),
    y=alt.Y('poids_bruite').scale(type='log', base=2),
    color=alt.Color('species_id'),
    facet=alt.Facet('plot_id').columns(5),
    tooltip=['species_id'],
).configure_mark(
    opacity=0.05,
).properties(
    width=90,
    height=60,
)

* Pour créer une grille de facettes telle qu'une variable change de
  valeur d'une rangée à l'autre et qu'une seconde variable change
  d'une colonne à l'autre, on utilisera les canaux `row` et `column` :

In [ ]:
# Garder uniquement quelques années
surveys2000 = surveys_complet[surveys_complet['year'].isin([2000, 2001, 2002])]

alt.Chart(surveys2000).mark_point().encode(
    x=alt.X('longueur_bruitee'),
    y=alt.Y('poids_bruite').scale(type='log', base=2),
    color=alt.Color('species_id'),
    row=alt.Row('sex'),
    column=alt.Column('year'),
    tooltip=['species_id'],
).configure_mark(
    opacity=0.05,
).properties(
    width=128,
    height=128,
)

### Exercice - Facettes
* Créez deux facettes selon le `sex`
* Chaque facette aura :
  * Les années en axe des x
  * Le poids moyen en axe des y
  * Une courbe de couleur par espèce

(5 min.)

In [ ]:
alt.Chart(surveys_complet).mark_line().encode(
    x=alt.X('year').type('ordinal'),
    y=alt.Y('weight').aggregate('mean'),
    color=alt.Color('species_id'),
    facet=alt.Facet('sex'),
).properties(
    width=256,
)

## Visualiser des distributions
* Tout d'abord, un box-plot peut être utilisé :

In [ ]:
alt.Chart(surveys_complet).mark_boxplot().encode(
    x=alt.X('species_id').title("Identifiant d'espèce"),
    y=alt.Y('poids_bruite').scale(type='log', base=2).title('Poids (g)'),
    color=alt.Color('species_id').legend(None),
)

* Des facettes étroites peuvent être utilisées pour afficher
  plusieurs nuages de points :

In [ ]:
alt.Chart(surveys_complet).transform_calculate(
    bruit='random()-0.5'  # Position horizontale dans la facette
).mark_circle(size=4).encode(
    x=alt.X('bruit').type('quantitative').axis(None).title(None),
    y=alt.Y('poids_bruite').scale(type='log', base=2).title('Poids (g)'),
    color=alt.Color('species_id').legend(None),
    column=alt.Column('species_id').title('Poids selon les espèces'),
).configure_mark(
    opacity=0.25,  # Transparence des mark_circle()
).configure_facet(
    spacing=0,     # Supprimer la marge entre les facettes
).configure_view(
    stroke=None,   # Enlever la boîte autour des facettes
).properties(
    width=18,      # Largeur des facettes
)

### Exercice - Distributions
Pour cet exercice, on cherche à avoir les vrais noms d'espèce sur
l'axe horizontal d'un box-plot.

`1`. Recalculez la jonction de gauche entre `surveys_complet` et
     le détail des espèces dans `species.csv`. (3 min.)

In [ ]:
species_df = pd.read_csv('../data/species.csv')

jonc_gauche = pd.merge(
    left=surveys_complet, right=species_df,
    on='species_id', how='left')

jonc_gauche.columns

`2`. Créez le box-plot:
* Le nom des espèces sur l'axe horizontal, avec l'étiquette "Espèce"
* Le poids bruité dans l'axe vertical, selon une échelle logarithmique
  en base 2 et avec l'étiquette "Poids (g)"
* Une couleur selon l'identifiant d'espèce
* Un titre pour le graphique

(6 min.)

In [ ]:
alt.Chart(jonc_gauche).mark_boxplot().encode(
    x=alt.X('species').title('Espèce'),
    y=alt.Y('poids_bruite').scale(type='log', base=2).title('Poids (g)'),
    color=alt.Color('species_id').legend(None),
).properties(
    title='Distribution des poids par espèce',
)

## Résumé technique
* **Module Altair**
  * `import altair as alt`
  * Désactiver limite : `alt.data_transformers.disable_max_rows()`
* **Création d'un graphique vierge**
  * `graphique = alt.Chart(df)`
* **Choix du type de marqueurs** à afficher
  * `graphique.mark_point()`
  * `graphique.mark_bar()`
  * `graphique.mark_line()`
  * `graphique.mark_boxplot()`
  * `graphique.mark_circle(size=N)`
  * `graphique.mark_area(orient='horizontal')`
* **Assigner des variables** à des canaux du graphique
  * `graphique.encode(...)`
  * Différents canaux :
    * `x=alt.X('varX')` et `y=alt.Y('varY')`
      * `.type('type')`, avec les
        [différents types](https://altair-viz.github.io/user_guide/encodings/index.html#encoding-data-types) :
        * Quantités continues : `'quantitative'`, `'var:Q'`
        * Quantités discrètes triées : `'ordinal'`, `'var:O'`
        * Catégories discrètes : `'nominal'`, `'var:N'`
        * Dates et heures : `'temporal'`, `'var:T'`
      * `.aggregate('stat')`,
        avec les statistiques `'mean'`, `'median'`, etc.
      * `.scale(type='log', base=2)`
      * `.title('Nom axe X ou Y')`
    * `color=alt.Color('varCouleur')`
      * `.legend(None)`
      * `.scale(domain=[...], range=['#114499', ...])`
    * `facet=alt.Facet('varFacet')`
      * `.columns(N)`
    * `row=alt.Row('varRangées')`
    * `column=alt.Column('varColonne')`
    * `tooltip=['varX', 'varY', 'varCouleur', ...]`
* **Autres propriétés du graphique**
  * `graphique.interactive()`
  * `graphique.configure_mark(opacity=0.05)`
  * `graphique.properties(...)`
    * `width=400`
    * `height=300`
    * `title='Titre'`
  * `graphique.configure_facet(spacing=0)`
  * `graphique.configure_view(stroke=None, width=20)`
* **Sauvegarde**
  * `graphique.save("graphique.html")`
  * `graphique.save("graphique.png")`